
# Citation and Conference Data Join

**TODO**

Jupyter Notebook for web scraping of the missing conferences locations of the DBLP dump.

For this process, the following CSV file is needed: ```out_dblp_papers_and_locations.csv```. <br>
The above file must be generated running the ```1 - dblp_add_locations_from_raw_dblp_dump.ipynb``` Notebook that is contained in the same folder as this Noteook.

In particular, the following operations are going to be executed:
* Opening of the CSV peprocessed dump
* Obtaining the missing locations with queries to the DBLP website
* Fix of the locations format

Lastly, the entire preprocessed dump is going to be saved on disk in CSV format

In [2]:
# Libraries Import
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

## File Paths
Please set your working directory paths.

In [3]:
# ******************* PATHS ********************+

# Dumps Directory Path
path_file_import = r'/Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/Import/'

# CSV Exports Directory Path
path_file_export = r'/Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/Export/'

## Read of the CSV Preprocessed Dumps

In [4]:
df_coci = pd.read_csv(path_file_export + 'out_coci_citations_count.csv', low_memory=False, index_col=[0])
df_coci

,article,citations_count
0,10.10.18045/zbefri.2015.2.207,1
1,10.1000/182,3
2,10.1000/res#test,1
3,10.1001,6
4,10.1001/.391,22
...,...,...
58110410,10.9799/ksfan.2017.30.4.728,1
58110411,10.9799/ksfan.2017.30.5.1035,1
58110412,10.9799/ksfan.2018.31.1.033,1
58110413,10.9799/ksfan.2019.32.4.328,1


In [ ]:
df_dblp = pd.read_csv(path_file_export + 'out_dblp_papers_and_locations_final.csv', low_memory=False, index_col=[0])
df_dblp

In [5]:
df_mag = pd.read_csv(path_file_export + 'out_mag_citations_and_locations_final.csv', low_memory=False, index_col=[0])
df_mag.drop(df_mag.filter(regex="Unname"), axis=1, inplace=True)
df_mag

,CitationCount,ConferenceLocation,ConferenceNormalizedName,ConferenceSeriesDisplayName,ConferenceSeriesNormalizedName,Doi,EstimatedCitation,OriginalTitle,PaperTitle,Year
0,12,"Austin, TX",disc 2014,International Symposium on Distributed Computing,DISC,10.1007/978-3-662-45174-8_28,12,The Adaptive Priority Queue with Elimination a...,the adaptive priority queue with elimination a...,2014
1,10,"Wrocław, Poland",esa 2014,European Symposium on Algorithms,ESA,10.1007/978-3-662-44777-2_60,10,Document Retrieval on Repetitive Collections,document retrieval on repetitive collections,2014
2,20,"Innsbruck, Austria",enter 2013,Information and Communication Technologies in ...,ENTER,10.1007/978-3-319-03973-2_13,20,SoCoMo Marketing for Travel and Tourism,socomo marketing for travel and tourism,2013
3,0,"Provence, France",dexa 2002,Database and Expert Systems Applications,DEXA,10.1007/3-540-46146-9_77,0,Similarity Image Retrieval System Using Hierar...,similarity image retrieval system using hierar...,2002
4,19,"Zakopane, Poland",icaisc 2006,International Conference on Artificial Intelli...,ICAISC,10.1007/11785231_94,19,Leukemia prediction from gene expression data—...,leukemia prediction from gene expression data ...,2006
...,...,...,...,...,...,...,...,...,...,...
4409807,0,Singapore,iecon 2020,Conference of the Industrial Electronics Society,IECON,10.1109/IECON43393.2020.9254316,0,Loss Reduction by Synchronous Rectification in...,loss reduction by synchronous rectification in...,2020
4409808,0,"Paris, France",bmsb 2020,International Symposium on Broadband Multimedi...,BMSB,10.1109/BMSB49480.2020.9379806,0,Data Over Cable Services – Improving the BICM ...,data over cable services improving the bicm ca...,2020
4409809,0,NaN,acc 1988,American Control Conference,ACC,10.1109/ACC.1988.4172843,0,Model Reference Robust Adaptive Control withou...,model reference robust adaptive control withou...,1988
4409810,0,"Orlando, Florida, USA",icassp 2002,"International Conference on Acoustics, Speech,...",ICASSP,10.1109/ICASSP.2002.1005676,0,Missing data speech recognition in reverberant...,missing data speech recognition in reverberant...,2002


## Preparation of the CSV Preprocessed COCI Dump

Order by citations count descending to see the articles with the most citations

In [11]:
# Order by citations count descending to see the articles with the most citations
df_coci = df_coci.sort_values(by='citations_count', ascending=False)
df_coci.iloc[:5]

,article,citations_count
18885082,10.1016/s0021-9258(19)52451-6,200934
23727779,10.1038/227680a0,181317
9251667,10.1016/0003-2697(76)90527-3,167747
33791759,10.1103/physrevlett.77.3865,114365
3217089,10.1006/meth.2001.1262,101209


Renaming the article column to doi and making sure that everything is in lowercase:

In [12]:
df_coci = df_coci.rename(columns={'article': 'Doi'})
df_coci = df_coci.reindex(sorted(df_coci.columns), axis=1)

df_coci.doi = df_coci.doi.str.lower()
df_coci.iloc[:5]

AttributeError: 'DataFrame' object has no attribute 'doi'

## Preparation of the CSV Preprocessed DBLP Dump

### Removing the DOI Website Path from the DOI URL (ee Column)

First we need to fix the rows that have multiple URLs separated by the | character

In [ ]:
for dblp_index, dblp_row in df_dblp.iterrows():

    if str(dblp_row['ee']).split('|').__len__() >= 2:
        df_dblp.at[dblp_index, 'ee'] = dblp_row['ee'].split('|')[1]

Removing the DOI website path

In [ ]:
df_dblp['doi'] = np.nan
df_dblp.doi = df_dblp.ee.str.lower().str.split('.org/').str[1]

# Column sort
df_dblp = df_dblp.reindex(sorted(df_dblp.columns), axis=1)

df_dblp.loc[:5]

### Creation of the DBLP Conference Instance Identifier
Before we can proceed with the join operations, we first need to create a conference identifier that will allow us to join the conference data between DBLP and MAG.

The chosen identifier is going to be composed in the same way of the ConferenceNormalizedName field of the MAG dump: name of the conference series (lower case) + year.

#### Extraction of the Conference Series Name from the Column "key"

In [ ]:
df_dblp['ConferenceNormalizedName'] = np.nan
df_dblp.ConferenceNormalizedName = df_dblp.key.str.split('/').str[1]

# Column sort
df_dblp = df_dblp.reindex(sorted(df_dblp.columns), axis=1)
df_dblp.loc[:5]

#### Fix of the Wrong Conferences Series Names
The above operation works for the most of the rows. However, there are some special cases that we need to fix manually.

Fix of the papers that need the extraction from the crossref field:

In [ ]:
crossref_list = ['agp', 'aipl', 'amcc', 'amec', 'bcd', 'biotec', 'calculemus', 'cf', 'cnps', 'corr', 'cw', 'daglib', 'dali', 'dbtel', 'ecoopwException', 'express', 'f-egc', 'icsengt', 'ifip5-5', 'infinity', 'isdt', 'iwlcs', 'npiv', 'wcit', 'isdt', 'dsai', 'aipl', 'dbtel', 'cw']

Special cases:

In [ ]:
print_counter = 0

for dblp_index, dblp_row in df_dblp.iterrows():

    print_counter += 1
    if print_counter == 25000:
        print(f"Riga numero {dblp_index + 1}")
        print_counter = 0

    fixed = False

    for i in range (0, crossref_list.__len__()):
        if str(dblp_row['crossref']).split('/').__len__() >= 2:
            if crossref_list[i] == str(dblp_row['crossref']).split('/')[1]:
                df_dblp.at[dblp_index, 'ConferenceNormalizedName'] = crossref_list[i]
                fixed = True
                break
    
    if not fixed:
        if "ecoopwException" in str(dblp_row['crossref']):
            df_dblp.at[dblp_index, 'ConferenceNormalizedName'] = "ecoopw"
        elif "icsengt" in str(dblp_row['crossref']):
            df_dblp.at[dblp_index, 'ConferenceNormalizedName'] = "icset"

        if "ali2" in str(dblp_row['key']):
            df_dblp.at[dblp_index, 'ConferenceNormalizedName'] = "ali"
        elif "ictcs2" in str(dblp_row['key']):
            df_dblp.at[dblp_index, 'ConferenceNormalizedName'] = "ictcs" 


#### Adding the Year to the Conference Series Name

In [ ]:
df_dblp.ConferenceNormalizedName = df_dblp.ConferenceNormalizedName + ' ' + df_dblp.year.astype(str)

In [ ]:
df_dblp.loc[:5]

### Write of the Join Ready DBLP Dump on Disk

In [ ]:
# Write of the resulting CSV on Disk
df_dblp.to_csv(path_file_export + 'out_dblp_join_ready.csv')
print(f'Successfully Exported the Processed CSV to {path_file_export}out_dblp_join_ready.csv')

## Join Between DBLP and MAG

Rename of the DBLP doi column:

In [7]:
df_dblp = df_dblp.rename(columns={'doi': 'Doi'})

Now we can proceed with the join and cleaning operations:

In [8]:
df_mag = pd.merge(df_mag, df_dblp, on=['Doi'], how='outer')

# Combine the conference location columns
df_mag['ConferenceLocation_x'] = df_mag['ConferenceLocation_x'].fillna(df_mag['ConferenceLocation_y'])
df_mag.rename(columns = {'ConferenceLocation_x':'ConferenceLocation'}, inplace=True)
df_mag = df_mag.drop(columns=['ConferenceLocation_y'])

# Combine the conference normalized name columns
df_mag['ConferenceNormalizedName_x'] = df_mag['ConferenceNormalizedName_x'].fillna(df_mag['ConferenceNormalizedName_y'])
df_mag.rename(columns = {'ConferenceNormalizedName_x':'ConferenceNormalizedName'}, inplace=True)
df_mag = df_mag.drop(columns=['ConferenceNormalizedName_y'])

# Combine the year columns
df_mag['Year'] = df_mag['Year'].fillna(df_mag['year'])
df_mag = df_mag.drop(columns=['year'])

# Drop of the DBLP columns that are not needed anymore
df_mag = df_mag.drop(columns=['crossref', 'ee', 'key', 'url'])

# Drop of the MAG columns that are not needed anymore
df_mag = df_mag.drop(columns=['PaperTitle', 'ConferenceSeriesDisplayName', 'ConferenceSeriesNormalizedName'])

# Rename of some columns to remove ambiguity
df_mag.rename(columns={'CitationCount': 'CitationCount_Mag', 'EstimatedCitation': 'CitationCount_MagEstimated'}, inplace=True)

# Column sort
df_mag = df_mag.reindex(sorted(df_mag.columns), axis=1)

# Fix of the year column data type
df_mag = df_mag.astype({"Year": int}) 

df_mag

,CitationCount_Mag,CitationCount_MagEstimated,ConferenceLocation,ConferenceNormalizedName,ConferenceTitle,Doi,OriginalTitle,Year
0,12.0,12.0,"Austin, TX",disc 2014,Distributed Computing - 28th International Sym...,10.1007/978-3-662-45174-8_28,The Adaptive Priority Queue with Elimination a...,2014
1,10.0,10.0,"Wrocław, Poland",esa 2014,Algorithms - ESA 2014 - 22th Annual European S...,10.1007/978-3-662-44777-2_60,Document Retrieval on Repetitive Collections,2014
2,20.0,20.0,"Innsbruck, Austria",enter 2013,Information and Communication Technologies in ...,10.1007/978-3-319-03973-2_13,SoCoMo Marketing for Travel and Tourism,2013
3,0.0,0.0,"Provence, France",dexa 2002,"Database and Expert Systems Applications, 13th...",10.1007/3-540-46146-9_77,Similarity Image Retrieval System Using Hierar...,2002
4,19.0,19.0,"Zakopane, Poland",icaisc 2006,Artificial Intelligence and Soft Computing - I...,10.1007/11785231_94,Leukemia prediction from gene expression data—...,2006
...,...,...,...,...,...,...,...,...
6143709,NaN,NaN,"Thessaloniki, Greece",sapere 2011,Philosophy and Theory of Artificial Intelligen...,10.1007/978-3-642-31674-6_9,NaN,2011
6143710,NaN,NaN,"Thessaloniki, Greece",sapere 2011,Philosophy and Theory of Artificial Intelligen...,10.1007/978-3-642-31674-6_20,NaN,2011
6143711,NaN,NaN,"Thessaloniki, Greece",sapere 2011,Philosophy and Theory of Artificial Intelligen...,10.1007/978-3-642-31674-6_25,NaN,2011
6143712,NaN,NaN,"Thessaloniki, Greece",sapere 2011,Philosophy and Theory of Artificial Intelligen...,10.1007/978-3-642-31674-6_12,NaN,2011


### Write of the DBLP + MAG Dump on Disk

In [9]:
# Write of the resulting CSV on Disk
df_mag.to_csv(path_file_export + 'out_dblp_and_mag_joined.csv')
print(f'Successfully Exported the Preprocessed CSV to {path_file_export}out_dblp_and_mag_joined.csv')

Successfully Exported the Preprocessed CSV to /Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/Export/out_dblp_and_mag_joined.csv


## Join Between DBLP+MAG and COCI

In [13]:
df_mag = pd.merge(df_mag, df_coci, on=['Doi'], how='left')

df_mag.iloc[:5]

,CitationCount_Mag,CitationCount_MagEstimated,ConferenceLocation,ConferenceNormalizedName,ConferenceTitle,Doi,OriginalTitle,Year,citations_count
0,12.0,12.0,"Austin, TX",disc 2014,Distributed Computing - 28th International Sym...,10.1007/978-3-662-45174-8_28,The Adaptive Priority Queue with Elimination a...,2014,10.0
1,10.0,10.0,"Wrocław, Poland",esa 2014,Algorithms - ESA 2014 - 22th Annual European S...,10.1007/978-3-662-44777-2_60,Document Retrieval on Repetitive Collections,2014,5.0
2,20.0,20.0,"Innsbruck, Austria",enter 2013,Information and Communication Technologies in ...,10.1007/978-3-319-03973-2_13,SoCoMo Marketing for Travel and Tourism,2013,11.0
3,0.0,0.0,"Provence, France",dexa 2002,"Database and Expert Systems Applications, 13th...",10.1007/3-540-46146-9_77,Similarity Image Retrieval System Using Hierar...,2002,1.0
4,19.0,19.0,"Zakopane, Poland",icaisc 2006,Artificial Intelligence and Soft Computing - I...,10.1007/11785231_94,Leukemia prediction from gene expression data—...,2006,9.0


Column rename and sort:

In [14]:
df_mag.rename(columns={'citations_count': 'CitationCount_COCI'}, inplace=True)
df_mag = df_mag.reindex(sorted(df_mag.columns), axis=1)
df_mag

,CitationCount_COCI,CitationCount_Mag,CitationCount_MagEstimated,ConferenceLocation,ConferenceNormalizedName,ConferenceTitle,Doi,OriginalTitle,Year
0,10.0,12.0,12.0,"Austin, TX",disc 2014,Distributed Computing - 28th International Sym...,10.1007/978-3-662-45174-8_28,The Adaptive Priority Queue with Elimination a...,2014
1,5.0,10.0,10.0,"Wrocław, Poland",esa 2014,Algorithms - ESA 2014 - 22th Annual European S...,10.1007/978-3-662-44777-2_60,Document Retrieval on Repetitive Collections,2014
2,11.0,20.0,20.0,"Innsbruck, Austria",enter 2013,Information and Communication Technologies in ...,10.1007/978-3-319-03973-2_13,SoCoMo Marketing for Travel and Tourism,2013
3,1.0,0.0,0.0,"Provence, France",dexa 2002,"Database and Expert Systems Applications, 13th...",10.1007/3-540-46146-9_77,Similarity Image Retrieval System Using Hierar...,2002
4,9.0,19.0,19.0,"Zakopane, Poland",icaisc 2006,Artificial Intelligence and Soft Computing - I...,10.1007/11785231_94,Leukemia prediction from gene expression data—...,2006
...,...,...,...,...,...,...,...,...,...
6143709,4.0,NaN,NaN,"Thessaloniki, Greece",sapere 2011,Philosophy and Theory of Artificial Intelligen...,10.1007/978-3-642-31674-6_9,NaN,2011
6143710,4.0,NaN,NaN,"Thessaloniki, Greece",sapere 2011,Philosophy and Theory of Artificial Intelligen...,10.1007/978-3-642-31674-6_20,NaN,2011
6143711,2.0,NaN,NaN,"Thessaloniki, Greece",sapere 2011,Philosophy and Theory of Artificial Intelligen...,10.1007/978-3-642-31674-6_25,NaN,2011
6143712,NaN,NaN,NaN,"Thessaloniki, Greece",sapere 2011,Philosophy and Theory of Artificial Intelligen...,10.1007/978-3-642-31674-6_12,NaN,2011


## Converting the NaN Citations to 0

In [17]:
df_mag['CitationCount_COCI'] = df_mag['CitationCount_COCI'].fillna(0)
df_mag['CitationCount_Mag'] = df_mag['CitationCount_Mag'].fillna(0)
df_mag['CitationCount_MagEstimated'] = df_mag['CitationCount_MagEstimated'].fillna(0)

Fix of the data type:

In [18]:
df_mag = df_mag.astype({"CitationCount_COCI": int}) 
df_mag = df_mag.astype({"CitationCount_Mag": int}) 
df_mag = df_mag.astype({"CitationCount_MagEstimated": int}) 

In [19]:
df_mag

,CitationCount_COCI,CitationCount_Mag,CitationCount_MagEstimated,ConferenceLocation,ConferenceNormalizedName,ConferenceTitle,Doi,OriginalTitle,Year
0,10,12,12,"Austin, TX",disc 2014,Distributed Computing - 28th International Sym...,10.1007/978-3-662-45174-8_28,The Adaptive Priority Queue with Elimination a...,2014
1,5,10,10,"Wrocław, Poland",esa 2014,Algorithms - ESA 2014 - 22th Annual European S...,10.1007/978-3-662-44777-2_60,Document Retrieval on Repetitive Collections,2014
2,11,20,20,"Innsbruck, Austria",enter 2013,Information and Communication Technologies in ...,10.1007/978-3-319-03973-2_13,SoCoMo Marketing for Travel and Tourism,2013
3,1,0,0,"Provence, France",dexa 2002,"Database and Expert Systems Applications, 13th...",10.1007/3-540-46146-9_77,Similarity Image Retrieval System Using Hierar...,2002
4,9,19,19,"Zakopane, Poland",icaisc 2006,Artificial Intelligence and Soft Computing - I...,10.1007/11785231_94,Leukemia prediction from gene expression data—...,2006
...,...,...,...,...,...,...,...,...,...
6143709,4,0,0,"Thessaloniki, Greece",sapere 2011,Philosophy and Theory of Artificial Intelligen...,10.1007/978-3-642-31674-6_9,NaN,2011
6143710,4,0,0,"Thessaloniki, Greece",sapere 2011,Philosophy and Theory of Artificial Intelligen...,10.1007/978-3-642-31674-6_20,NaN,2011
6143711,2,0,0,"Thessaloniki, Greece",sapere 2011,Philosophy and Theory of Artificial Intelligen...,10.1007/978-3-642-31674-6_25,NaN,2011
6143712,0,0,0,"Thessaloniki, Greece",sapere 2011,Philosophy and Theory of Artificial Intelligen...,10.1007/978-3-642-31674-6_12,NaN,2011


## Write of the Final CSV on Disk

Saving the resulting dataframe on disk in CSV format.

In [20]:
# Write of the resulting CSV on Disk
df_mag.to_csv(path_file_export + 'out_citations_and_conferences.csv')
print(f'Successfully Exported the Processed CSV to {path_file_export}out_citations_and_conferences.csv')

Successfully Exported the Processed CSV to /Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/Export/out_citations_and_conferences.csv


Check of the Exported CSV to be sure that everything went fine.

In [21]:
# Check of the Exported CSV
df_joined_exported_csv = pd.read_csv(path_file_export + 'out_citations_and_conferences.csv', low_memory=False, index_col=[0])
df_joined_exported_csv

,CitationCount_COCI,CitationCount_Mag,CitationCount_MagEstimated,ConferenceLocation,ConferenceNormalizedName,ConferenceTitle,Doi,OriginalTitle,Year
0,10,12,12,"Austin, TX",disc 2014,Distributed Computing - 28th International Sym...,10.1007/978-3-662-45174-8_28,The Adaptive Priority Queue with Elimination a...,2014
1,5,10,10,"Wrocław, Poland",esa 2014,Algorithms - ESA 2014 - 22th Annual European S...,10.1007/978-3-662-44777-2_60,Document Retrieval on Repetitive Collections,2014
2,11,20,20,"Innsbruck, Austria",enter 2013,Information and Communication Technologies in ...,10.1007/978-3-319-03973-2_13,SoCoMo Marketing for Travel and Tourism,2013
3,1,0,0,"Provence, France",dexa 2002,"Database and Expert Systems Applications, 13th...",10.1007/3-540-46146-9_77,Similarity Image Retrieval System Using Hierar...,2002
4,9,19,19,"Zakopane, Poland",icaisc 2006,Artificial Intelligence and Soft Computing - I...,10.1007/11785231_94,Leukemia prediction from gene expression data—...,2006
...,...,...,...,...,...,...,...,...,...
6143709,4,0,0,"Thessaloniki, Greece",sapere 2011,Philosophy and Theory of Artificial Intelligen...,10.1007/978-3-642-31674-6_9,NaN,2011
6143710,4,0,0,"Thessaloniki, Greece",sapere 2011,Philosophy and Theory of Artificial Intelligen...,10.1007/978-3-642-31674-6_20,NaN,2011
6143711,2,0,0,"Thessaloniki, Greece",sapere 2011,Philosophy and Theory of Artificial Intelligen...,10.1007/978-3-642-31674-6_25,NaN,2011
6143712,0,0,0,"Thessaloniki, Greece",sapere 2011,Philosophy and Theory of Artificial Intelligen...,10.1007/978-3-642-31674-6_12,NaN,2011
